In [1]:
%run StdPackages.ipynb
d['rawData'] = os.path.join(d['data'],'rawData69') # add
d['processedData'] = os.path.join(d['data'],'processedData') # update to raw data folder
os.chdir(d['py'])
import IOfunctions_withoutDurables
from loadIO import *

No clean-up of work-folder


# IO data for Danish Economy from 1990-2020

This sets up balanced IO data for the Danish Economy from 1990-2020. We rely on the 69 sector split whereever possible, and use different imputations for the period 1990-1993 where data conventions were slightly different.

*Basic settings:*

In [2]:
ws = GpyDB.initWs(d['work']) # initialize a workspace so we don't have to do this every time we create a database
t0 = 1990 # initial year
tE = 2019 # terminal year
file_k = os.path.join(d['rawData'], 'Durables.xlsx') # xlsx file with data on durables
file_m = os.path.join(d['rawData'], 'Emissions.xlsx') # xlsx file with data on emissions

### 1. Data on durables

*Mapping of names:*

In [3]:
namesToInv = {'Boliger': '5110', 
              'Andre bygninger': '5121',
              'Anlæg': '5122',
              'Transportmidler': '5131',
              'ICT udstyr, andre maskiner og inventar samt våbensystemer': '513x',
              'Stambesætninger mv.': '5150',
              'Intellektuelle rettigheder': '517x'}
namesInvVariables = {'AN.11 Faste aktiver, nettobeholdning, primo året': 'K',
                     'P.51c Forbrug af fast realkapital': 'Depr1',
                     'P.51g Faste bruttoinvesteringer': 'I',
                     'K.3 Tab ved katastrofer': 'Depr2',
                     'K.7 Nominelle kapitalgevinster og -tab': 'Depr3',
                     'AN.11 Faste aktiver, nettobeholdning ultimo året': 'Kp1'}

Durables data:

In [4]:
dur = getDurables(file_k, 'NABK69', namesToInv, namesInvVariables)

### 2. Read emissions data

In [5]:
emissions = emissionsReadData(file_m) # NB: This rescales emission taxes to 1000 DKK - as this is the standard unit used for taxes in the main IO table

Emissions on sectoral level is defined only for "emissions from the Danish economy" and includes things like international shipping and does not include the LULUCF sector. Also, we only include emissions from the production sectors (and not direct emissions e.g. from households). This scales emissions such that we follow the "inclusive LULUCF"-like measure that CO2 targets are formulated in terms of:

In [6]:
emissions['qCO2'] = emissions['qCO2'] * (emissions['totalEmissions'] / emissions['qCO2'].groupby('t').sum())

### 3. IO data

IO data is a bit more complicated, as they are not naturally arranged as time series (each year is a large matrix). As we already have IO data functions available that deals with the data year-by-year, we loop over the relevant years ad extract and adjust relevant data along the way.

*Note: Data on durables uses units mio. DKK, whereas the rest of the IO data uses 1000 DKK. Here, we rescale to make everything in 100 mio DKK.*

*The standard settings for doing this with NR69 detailed dataset in English:*

In [7]:
kwargs_v = {} 
kwargs_v['rowMarkers'] = {'P': {'ref': 'Danish production', 'offset': {}},
                          'M': {'ref': 'Imports', 'offset': {}},
                          'OT':{'ref': 'Other Foreign Transactions', 'offset': {}},
                          'PI':{'ref': 'Primary Factors', 'offset':{}},
                          'TI':{'ref': 'Input / final demand, purchasers prices', 'offset':{}},
                          'PV':{'ref': 'Total Output', 'offset': {}}
                         }
kwargs_v['colMarkers'] = {'In': {'ref': 'Input in production (Transaction code 2000)', 'offset': {'colE':-7}},
                          'C' : {'ref': 'GFCF', 'offset': {'col0': -5, 'colE': -5}},
                          'G_NPISH': {'ref': 'GFCF', 'offset': {'col0': -4, 'colE': -4}},
                          'G_MVPC':  {'ref': 'GFCF', 'offset': {'col0': -3, 'colE': -3}},
                          'G_NMVPC': {'ref': 'GFCF', 'offset': {'col0': -2, 'colE': -2}},
                          'G_CPC':   {'ref': 'GFCF', 'offset': {'col0': -1, 'colE': -1}},
                          'I': {'ref': 'GFCF', 'offset':{}},
                          'Other': {'ref': 'Other uses', 'offset': {}},
                          'T': {'ref': 'Total'}}
kwargs_v['category'] = {'taxCategories': ['Product taxes (excl. VAT)', 'VAT', 'Other production taxes'],
                        'wageCategory' : 'Wages and Salaries',
                        'residualIncomeCategory': 'Gross Surplus and mixed income',
                        'itoryCategories': ['5300','5200'],
                        'exportCategory': '6000'}
kwargs_i = {}
kwargs_i['rMarker'] = 'Total investment, purchase prices'
kwargs_i['cMarkers'] = ['Investing industries', 'Total']

*Loop through years and collect data:*

In [8]:
dbs = {}
for t in range(t0, tE+1):
    # Load data:
    name = f'IO{t}_full'
    file_v = os.path.join(d['rawData'], f'InputOutput_{t}.xlsx')
    I = IOfunctions_withoutDurables.readIO(name = name, ws = ws, file_v = file_v, kwargs_v = kwargs_v)
    I() # go through standard methods to extract and define data
    addDurablesToDb(I.db, dur, t) # add durables data
    addEmissionsToDb(I.db, emissions, t) # add emissions data
    # investments and durables are measured in mio DKK, the other io data in 1000 DKK; this adjusts this:
    [I.db.__setitem__(k, I.db(k)/1000) for k in ('vTax','TotalTax','vD','vC','vC_tax')];
    I.cleanOtherForeignTransactions() # Clean up other foreign transactions
    AggDB.readSets(I.db) # add sets
    I.db['n'] = adj.rc_pd(I.db('n'), ('not', I.db('n_Fother'))) # Clean up
    del(I.db.series['n_Fother'])
    [I.db.__setitem__(k,stdSort(v.vals)) for k,v in I.db.getTypes(['var','par']).items()]; # Sort indices:
    dbs[t] = I.db

*Create aggregate database with $t$ index. Note: This operation assumes that definition of symbols remain constant over time. For instant:*
* Set elements remain the same (for instance, sectors do not change names over time),
* Subset and mappings remain the same over time: We do not add the time index to these, but instead assume that we can simply copy the symbols from the latest database.


In [9]:
db = GpyDB(name = f'IO69', ws = ws, data_folder = d['processedData'], alias = [('n','nn'), ('n','nnn'), ('s','ss')]) # initialize agg database
[db.__setitem__(var, concatVar(dbs, var)) for var in dbs[tE].getTypes(['var','par'])]; # concatenate variables
[db.__setitem__(s, dbs[tE](s)) for s in dbs[tE].getTypes(['subset','map'])]; # copy subsets and mappings from the latest database
AggDB.readSets(db) # read fundamental sets from the symbols in variables and parameters 

Export database:

In [10]:
db.export()